# --------------------------------------------------------------------------------------------- 
                                                                                                
# -----  Segmenting and Clustering neighbourhoods in Toronto   -----                                                                                              
# ---------------------------------------------------------------------------------------------

## First, import all relevant libraries ( I had to install several as i ran the notebook locally on my computer)

In [1]:
import numpy as np

import sys
!{sys.executable} -m pip install pandas 

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Could not build wheels for pandas, since package 'wheel' is not installed.
Could not build wheels for pytz, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.


In [3]:
#!{sys.executable} -m pip install geopy
from geopy.geocoders import Nominatim

#!{sys.executable} -m pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

!{sys.executable} -m pip install sklearn
from sklearn.cluster import KMeans


  Using cached sklearn-0.0.tar.gz (1.1 kB)
Could not build wheels for sklearn, since package 'wheel' is not installed.
Could not build wheels for scikit-learn, since package 'wheel' is not installed.
Could not build wheels for joblib, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for scipy, since package 'wheel' is not installed.
    Running setup.py install for sklearn: started
    Running setup.py install for sklearn: finished with status 'done'


ERROR: Invalid requirement: 'folium=0.5.0'
Hint: = is not a valid operator. Did you mean == ?


ModuleNotFoundError: No module named 'folium'

In [4]:
!{sys.executable} -m pip install folium==0.5.0
import folium

Could not build wheels for folium, since package 'wheel' is not installed.
Could not build wheels for jinja2, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.
Could not build wheels for MarkupSafe, since package 'wheel' is not installed.
    Running setup.py install for folium: started
    Running setup.py install for folium: finished with status 'done'


In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


In [6]:
import json

In [18]:
!{sys.executable} -m pip install bs4
from bs4 import BeautifulSoup

Could not build wheels for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'


In [23]:
!{sys.executable} -m pip install lxml

## Now, we obtain the dataset from the Wikipedia page using the BeautifulSoup library


In [47]:
wikiurl='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res=requests.get(wikiurl).text
soup = BeautifulSoup(res, 'html.parser')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

# create a new DataFrame from the three lists
toronto_neighbourhoods = pd.DataFrame({"Postal Code": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_neighbourhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,
1,M2A\n,Not assigned\n,
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront


## Now we have scraped the data from the wikipedia page and saved it in a pandas dataframe, we can start preprocessing

### Dropping rows where 'Borough' = 'Not Assigned'

In [48]:
# First, we must clean the first two columns, where "\n" has been appended to each variable during data scraping

toronto_neighbourhoods['Postal Code'] = toronto_neighbourhoods['Postal Code'].astype(str).str[:-1]
toronto_neighbourhoods['Borough'] = toronto_neighbourhoods['Borough'].astype(str).str[:-1]
toronto_neighbourhoods.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [49]:
#Now we can drop the rows where 'Borough'='Not Assigned'
toronto_neighbourhoods_clean = toronto_neighbourhoods[toronto_neighbourhoods.Borough != "Not assigned"].reset_index(drop=True)
toronto_neighbourhoods_clean.head()



,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### The original table on the wikipedia page allready has the neighbourhoods grouped per borough, though they are separated by "/". We shall replace these with ","

In [50]:
toronto_neighbourhoods_clean['Neighborhood']=toronto_neighbourhoods_clean['Neighborhood'].str.replace('/',',')
toronto_neighbourhoods_clean.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### We replace unassigned neighbourhoods with the Borough name (Although from inspection of the original dataset on the wikipedia page, there seem to be no defined boroughs with unassigned neighbourhoods):

In [52]:
 
count=0 # counter, to detect how many neighbourhoods had their name replaced
for index, row in toronto_neighbourhoods_clean.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        count=change+1
toronto_neighbourhoods_clean
#print(count, 'unassigned neighbourhoods were assigned their borough name')

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### We use the method shape() to check the size of the dataset:

In [54]:
toronto_neighbourhoods_clean.shape

(103, 3)

### We use the file available on the capstone project assignment instructions 

In [62]:
#Read local file downloaded onto computer
csv_loc = 'C:/Users/Terminal/Documents/Coursera/Capstone Project/Geospatial_Coordinates.csv'
coordinates = pd.read_csv(csv_loc)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [70]:
#Merge neighbourhood and coordinate dataframes. For convenience, the dataframe is renamed 'toronto_neighbourhoods_data'
Toronto_neighbourhoods_merge=pd.merge(toronto_neighbourhoods_clean, coordinates, on='Postal Code')
toronto_neighbourhoods_data=Toronto_neighbourhoods_merge
toronto_neighbourhoods_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Now, we create a map of toronto and superpose the coordinates of the neighbourhoods from the scraped data

### First we obtain coordinates for Toronto city, which we use later to center our folium map on

In [68]:
#First we obtain the latitude and longitude of Toronto city, to center our folium map on

address = 'Toronto'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of Toronto city:')
print('latitude',latitude) 
print('longitude',longitude)

Latitude and longitude of Toronto city:
latitude 43.6534817
longitude -79.3839347


In [71]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighbourhoods_data['Latitude'], toronto_neighbourhoods_data['Longitude'], toronto_neighbourhoods_data['Borough'], toronto_neighbourhoods_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

## Data exploration and clustering 

### As the assignment instructions suggest, we explore neighbourhoods in boroughs that include "Toronto"

In [212]:
# filter borough names that contain the word Toronto

#Alternative method
#mask = toronto_neighbourhoods_data['Borough'].str.contains('Toronto')    
#toronto_neighbourhoods_data[mask]

#Used method
Boroughs_Toronto=toronto_neighbourhoods_data[toronto_neighbourhoods_data['Borough'].str.contains("Toronto")]
print(Boroughs_Toronto.shape)
Boroughs_Toronto

(39, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259


In [82]:
#Neighbourhoods that contain "Toronto"
Boroughs_Toronto_list = Boroughs_Toronto[["Borough"]].values.ravel()
unique_values =  pd.unique(Boroughs_Toronto_list)
unique_values


array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

### Now we create a map of Toronto and the reduced list of neighbourhoods

In [213]:
# create map of Toronto using latitude and longitude values
toronto_map_new = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Boroughs_Toronto['Latitude'], Boroughs_Toronto['Longitude'], Boroughs_Toronto['Borough'], Boroughs_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map_new)  
    
toronto_map_new

### To explore the neighbourhoods, we use the FourSquare API 


In [84]:
CLIENT_ID = 'WD0HB4YZWDPF1I4OHH5UFHGPTRBR01PS4VWZOLHBFD4JOMUN' 
CLIENT_SECRET = 'RB3MHYRUBGZZVGZCSDM0ETR4OKAVZVGI03R5KEPW5NYLZQD4' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WD0HB4YZWDPF1I4OHH5UFHGPTRBR01PS4VWZOLHBFD4JOMUN
CLIENT_SECRET:RB3MHYRUBGZZVGZCSDM0ETR4OKAVZVGI03R5KEPW5NYLZQD4


### Following the New York clustering example from the course, we look for the top 100 venues in a radius of 500m of each neighbourhood

In [216]:
#Function to get top 100 venues within 500m of a given neighbourhood location
radius = 500
LIMIT = 100

#Define a list to store venue data from FourSquare
venues = []

#zip through all the roughs in the Boroughs_Toronto dataframe, appending venue data to the Venue list
for lat, long, post, borough, neighborhood in zip(Boroughs_Toronto['Latitude'], Boroughs_Toronto['Longitude'], Boroughs_Toronto['Postal Code'], Boroughs_Toronto['Borough'], Boroughs_Toronto['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [218]:
# convert the venues list into a new DataFrame
Toronto_venues_df = pd.DataFrame(venues)

# define the column names
Toronto_venues_df.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Borough Latitude', 'Borough Longitude', 'Venue Name', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

print(Toronto_venues_df.shape)
Toronto_venues_df.head()

(1623, 9)


,Postal Code,Borough,Neighborhood,Borough Latitude,Borough Longitude,Venue Name,Venue Latitude,Venue Longitude,Venue Category
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### Lets check how many venues were returned for each neighbourhood:


In [115]:
Toronto_venues_count = Toronto_venues_df.groupby('Neighborhood').count()
Toronto_venues_count = Toronto_venues_count.filter(['Neighborhood','Venue Name'])
Toronto_venues_count = Toronto_venues_count.rename(columns={'Venue Name': 'Venues'})
Toronto_venues_count.sort_values(by=['Venues'])
Toronto_venues_count

,Venues
Neighborhood,
Berczy Park,57
"Brockton , Parkdale Village , Exhibition Place",23
Business reply mail Processing CentrE,18
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17
Central Bay Street,63
Christie,17
Church and Wellesley,78
"Commerce Court , Victoria Hotel",100
Davisville,36


## Now we analyze each neighbourhood

In [193]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add postal code, borough and neighborhood column back to dataframe
toronto_onehot['Postal Code'] = Toronto_venues_df['Postal Code'] 
toronto_onehot['Borough'] = Toronto_venues_df['Borough'] 
toronto_onehot['Neighborhood'] = Toronto_venues_df['Neighborhood'] 

cols_to_move = ['Postal Code', 'Borough','Neighborhood']
toronto_onehot = toronto_onehot[ cols_to_move + [ col for col in toronto_onehot.columns if col not in cols_to_move ] ]

print(toronto_onehot.shape)
toronto_onehot.head()

(1623, 233)


,Postal Code,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

In [219]:
toronto_grouped = toronto_onehot.groupby(['Postal Code','Borough', 'Neighborhood']).mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(39, 233)


,Postal Code,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [242]:
toronto_grouped_by_hood = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.iloc[0,0])
toronto_grouped_by_hood.head()

M4E


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.000000,0.0,0.000000,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.035088,0.0,0.0,0.0,0.035088,0.0,0.0,0.0,0.000000,0.017544,0.052632,0.070175,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.0,0.000000,0.0,0.017544,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.017544,0.0,0.0,0.0,0.000000,0.017544,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.0,0.0,0.017544,0.017544,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000

### Let's print each neighborhood along with the top 5 most common venues

In [342]:
num_top_venues = 5


for hood in toronto_grouped_by_hood['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped_by_hood[toronto_grouped_by_hood['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.13
1  Breakfast Spot  0.09
2       Nightclub  0.09
3     Coffee Shop  0.09
4             Gym  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0           Yoga Studio  0.06
1                   Spa  0.06
2  Gym / Fitness Center  0.06
3         Garden Center  0.06
4                Garden  0.06


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4               Bar  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1                

### We proceed to generate a dataframe containing the previous data.

#### First, let's write a function to sort the venues in descending order.

In [304]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [374]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


In [365]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped_by_hood['Neighborhood']

for ind in np.arange(toronto_grouped_by_hood.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped_by_hood.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Bakery,Cheese Shop,Restaurant,Beer Bar,Pub,Portuguese Restaurant
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Nightclub,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
2,Business reply mail Processing CentrE,Yoga Studio,Auto Workshop,Light Rail Station,Skate Park,Smoke Shop,Burrito Place,Farmers Market,Spa,Fast Food Restaurant,Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Lounge,Airport Terminal,Airport,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Bar


## Clustering of neighbourhoods


### Now we run K-means to cluster the neighbourhoods


In [366]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped_by_hood.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 4, 1,
       1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [367]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = Boroughs_Toronto.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged # check the last columns!

(39, 16)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Burger Joint,Bank,Bar,Discount Store,Yoga Studio,Fried Chicken Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Cosmetics Shop,Department Store,Lingerie Store,Moroccan Restaurant,Italian Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Bakery,Cheese Shop,Restaurant,Beer Bar,Pub,Portuguese Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Bar
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Candy Store,Restaurant,Diner,Athletics & Sports,Italian Restaurant,Baby Store,Nightclub
30,M5H,Downtown Toronto,"Richmond , Adelaide , King",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Gym,Hotel,Thai Restaurant,Seafood Restaurant,Breakfast Spot
31,M6H,West Toronto,"Dufferin , Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Grocery Store,Pool,Liquor Store,Café,Recording Studio,Bar,Bank,Middle Eastern Restaurant


### Now we plot the neighbourhoods to visualize the different clusters K-Means has found 

In [368]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Let us examine the clusters that were returned by the K-means algorithm

### Cluster 1


In [369]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Restaurant,Café,Pub,Bakery,Pizza Place,Park,Japanese Restaurant,Jewelry Store


In [370]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Diner,Mexican Restaurant,Burger Joint,Bank,Bar,Discount Store,Yoga Studio,Fried Chicken Joint
9,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Restaurant,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel
15,Downtown Toronto,1,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Cosmetics Shop,Department Store,Lingerie Store,Moroccan Restaurant,Italian Restaurant
19,East Toronto,1,Health Food Store,Trail,Pub,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
20,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Bakery,Cheese Shop,Restaurant,Beer Bar,Pub,Portuguese Restaurant
24,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Salad Place,Fried Chicken Joint,Bubble Tea Shop,Burger Joint,Ice Cream Shop,Bar
25,Downtown Toronto,1,Grocery Store,Café,Park,Candy Store,Restaurant,Diner,Athletics & Sports,Italian Restaurant,Baby Store,Nightclub
30,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Gym,Hotel,Thai Restaurant,Seafood Restaurant,Breakfast Spot
31,West Toronto,1,Bakery,Pharmacy,Grocery Store,Pool,Liquor Store,Café,Recording Studio,Bar,Bank,Middle Eastern Restaurant


In [371]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge
61,Central Toronto,2,Park,Construction & Landscaping,Bus Line,Swim School,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
79,Central Toronto,2,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Indian Restaurant,Japanese Restaurant


In [372]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,Downtown Toronto,3,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Restaurant,Bakery,Yoga Studio,Pub,Beer Bar


In [373]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Central Toronto,4,Department Store,Sandwich Place,Park,Gym,Breakfast Spot,Hotel,Dog Run,Food & Drink Shop,Discount Store,Distribution Center


# Conclusion

From this relatively simple analysis (qualitative representation of venues in order of commonality), most neighbourhoods seem to fall into cluster 1, where coffe shops and cafes figure mainly in the 1st and 2nd most common venues, suggesting business areas, although there is one neighbourhood included that conyains in its 3 first most common venues an airport service, airport lounge and airport terminal, strongly suggesting it might be appropriate in a separate cluster. Cluster 1 remains unchanged even when K-Means is run for 5,6 and 7 clusters, so the correlation between data seems strong even if it doesnt seem so by visual inspection of the dataset. 

The second most numerous cluster is number 2, with 3 neighbourhoods, where  cafes and coffee shops seem to be less commoon but have a more important proportion of ethnic restaurants.

Clusters 0, 3 and 4 all contain only one neighbourhood. But a study in greater detail would be necessary to be able to discern the distinctive characteristics of all of them, possibly by studying the actual ammount of venues within the limits of each neighbourhood.
